In [ ]:
import showdown
import agent
import training
import torch
import json

In [ ]:
stateSize = 671

possibleActions = json.load(open("data/possible_actions.json", "r"))
actionSize = len(possibleActions)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent1 = agent.Agent(stateSize, actionSize, device, possibleActions)
#agent2 = agent.Agent(stateSize, actionSize, device, possibleActions)

sd1 = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "PoryAI-0", "Change This!", "wss://sim3.psim.us/showdown/websocket", "gen9randombattle", False, True, "", True)
#sd2 = showdown.Showdown("https://play.pokemonshowdown.com/action.php", "PoryAI-2", "password", "ws://localhost:8000/showdown/websocket", "gen9randombattle", False, False, "PoryAI-1")

await sd1.connectToShowdown()
#await sd2.connectNoSecurity()
trainer = training.Trainer([agent1], [sd1], stateSize, actionSize, 20000, 50)
await trainer.run()